In [10]:
import json
import requests

In [13]:
# Here we define our query as a multi-line string
query = '''
query MyQuery($id: Int) { # Define which variables will be used in the query (id)
  Media (id: $id, type: ANIME) { # Insert our variables into the query arguments (id) (type: ANIME is hard-coded in the query)
    id
    title {
      romaji
      english
      native
    }
  }
}
'''

# Define our query variables and values that will be used in the query request
variables = {
    'id': 15125
}

url = 'https://graphql.anilist.co'

# Make the HTTP Api request
response = requests.post(url, json={'query': query, 'variables': variables})

In [14]:
print(json.dumps(response.json(), indent=4))

{
    "data": {
        "Media": {
            "id": 15125,
            "title": {
                "romaji": "Teekyuu",
                "english": "Teekyuu",
                "native": "\u3066\u30fc\u304d\u3085\u3046"
            }
        }
    }
}


# Get Recommended Anime Given User

In [39]:
query = """
query CompletedRecs {
  MediaListCollection(userName:"SimpleCore", type:ANIME, status:COMPLETED, sort:SCORE_DESC) {
    lists {
      entries {
        score(format: POINT_100)
        media {
          id
          title {
            english
            romaji
          }
          recommendations(sort:RATING_DESC, page:0, perPage:10) {
            edges {
              node {
                rating
                mediaRecommendation {
                  id
                  title{
                    english
                    romaji
                  }
                }
              }
            }
          }
        }
      }
    }
  }
}
"""

In [40]:
response = requests.post(url, json={'query': query})

In [41]:
data = response.json()

In [61]:
from collections import defaultdict
from pprint import pprint
entries = data['data']['MediaListCollection']['lists'][0]['entries']
my_watched_anime = [(entry['media']['id'], entry['media']['title']['romaji']) for entry in entries]
my_watched_anime_keys = set(id for (id, name) in my_watched_anime)

In [70]:
recommendations_by_mode = defaultdict(int)
recommendations_by_weighted_mode = defaultdict(float)
recommendations_by_weighted_ratings = defaultdict(float)

for entry in entries:
    score = entry['score']
    if score < 7:
        continue
    entry_weight = score / 100
    for recommendation in entry['media']['recommendations']['edges']:
        media = recommendation['node']['mediaRecommendation']
        rating = recommendation['node']['rating']
        if media is None:
            continue
        if media['id'] not in my_watched_anime_keys:
            recommendations_by_mode[media['title']['romaji']] += 1
            recommendations_by_weighted_mode[media['title']['romaji']] += entry_weight
            recommendations_by_weighted_ratings[media['title']['romaji']] += entry_weight * rating
            
mode_recommendations = [(name, count) for name, count in recommendations_by_mode.items()]
mode_recommendations.sort(key=lambda recommendation: recommendation[1], reverse=True)
# pprint(mode_recommendations[:20])

weighted_mode_recommendations = [(name, count) for name, count in recommendations_by_weighted_mode.items()]
weighted_mode_recommendations.sort(key=lambda recommendation: recommendation[1], reverse=True)
# pprint(weighted_mode_recommendations[:20])

weighted_ratings_recommendations = [(name, count) for name, count in recommendations_by_weighted_ratings.items()]
weighted_ratings_recommendations.sort(key=lambda recommendation: recommendation[1], reverse=True)
pprint(weighted_ratings_recommendations[:50])

[('Vinland Saga', 1746.1999999999998),
 ('ONE PIECE', 1520.8),
 ('Dororo', 1419.25),
 ('Tate no Yuusha no Nariagari', 1073.05),
 ('Tokyo Revengers', 1024.0),
 ('Shinchou Yuusha: Kono Yuusha ga Ore TUEEE Kuse ni Shinchou Sugiru', 955.0),
 ('Promare', 899.0),
 ('Yuu☆Yuu☆Hakusho', 895.3),
 ('Black Clover', 835.0),
 ('NARUTO', 735.25),
 ('Summer Time Render', 720.1),
 ('Sono Bisque Doll wa Koi wo Suru', 700.7),
 ('Wotaku ni Koi wa Muzukashii', 698.8000000000001),
 ('Tonikaku Kawaii', 684.6),
 ('Wonder Egg Priority', 672.0),
 ('Re:Zero kara Hajimeru Isekai Seikatsu 2nd Season Part 2', 641.95),
 ('Josee to Tora to Sakanatachi', 641.1999999999999),
 ('Shin Seiki Evangelion', 638.9000000000001),
 ('Saiki Kusuo no Ψ-nan', 634.3),
 ('Kotonoha no Niwa', 629.05),
 ('Byousoku 5 Centimeter', 626.75),
 ('Hyouka', 624.2),
 ('Yuru Camp△', 569.65),
 ('Shiguang Dailiren', 550.3),
 ('Shinsekai yori', 548.65),
 ('Tensei Shitara Slime Datta Ken 2nd Season', 510.20000000000005),
 ('Zankyou no Terror', 498.25